<a href="https://colab.research.google.com/github/msamwelmollel/swahili-prompt-translation-for-swahili/blob/main/Uploading_prompts_to_be_translated_to_an_Argilla_Space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install huggingface_hub argilla datasets openai -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 

In [2]:
import json

In [13]:
SPACE_ID =  "DIBT-Swahili/prompt-translation-for-Swahili"
HOMEPAGE_URL = "https://dibt-swahili-prompt-translation-for-swahili.hf.space"
LANGUAGE = None, # i.e. "Swahili"

In [14]:
assert SPACE_ID and HOMEPAGE_URL and LANGUAGE, "Please set SPACE_ID and HOMEPAGE_URL to your space ID and homepage URL"

In [15]:
from huggingface_hub import space_info

assert space_info(SPACE_ID).runtime.storage.get("current") == "small", "Please ensure you have setup persistent storage for your space. Please see steps above"

In [16]:
import argilla as rg

OWNER_API_KEY = "b6f4a5b1-1c74-4927-b876-34ffe941706d" # if you haven't setup the secret this is the default owner api key
assert OWNER_API_KEY is not None, "Please set OWNER_API_KEY to the API token you just set in the Space settings"

rg.init(api_url=HOMEPAGE_URL, api_key=OWNER_API_KEY)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [17]:
from datasets import load_dataset

# load the dataset from the Hub
ds = load_dataset('DIBT/prompts_ranked_multilingual_benchmark')

In [18]:
# create the dataset with a pre-built template
argilla_ds = rg.FeedbackDataset.for_translation(
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
argilla_ds

FeedbackDataset(
   fields=[TextField(name='source', title='Source', required=True, type='text', use_markdown=True)]
   questions=[TextQuestion(name='target', title='Target', description='Translate the text.', required=True, type='text', use_markdown=True)]
   guidelines=This is a translation dataset that contains texts. Please translate the text in the text field.)
   metadata_properties=[])
   vectors_settings=[])
)

In [19]:
# create records
records = []
for row in ds["train"]:
    record = rg.FeedbackRecord(
        fields={"source": row["prompt"]},
        metadata=json.loads(row["metadata"]),
        external_id=row["row_idx"],
    )
    records.append(record)

In [20]:
# add records to the dataset
argilla_ds.add_records(records)

In [21]:
# push the dataset to Argilla
argilla_ds.push_to_argilla(f"DIBT Translation for {LANGUAGE}", workspace="admin")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/argilla/client/feedback/dataset/local/mixins.py:447 in   │
│ _create_argilla_dataset_or_raise                                                                 │
│                                                                                                  │
│    444 │   │   )                                                                                 │
│    445 │                                                                                         │
│    446 │   try:                                                                                  │
│ ❱  447 │   │   return datasets_api_v1.create_dataset(                                            │
│    448 │   │   │   client=httpx_client,                                                          │
│    449 │   │   │   name=name,                                                                    │
│    450 │   │   │   workspace_id=workspace.id,                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/argilla/client/sdk/v1/datasets/api.py:69 in              │
│ create_dataset                                                                                   │
│                                                                                                  │
│    66 │   │   response_obj = Response.from_httpx_response(response)                              │
│    67 │   │   response_obj.parsed = FeedbackDatasetModel(**response.json())                      │
│    68 │   │   return response_obj                                                                │
│ ❱  69 │   return handle_response_error(response)                                                 │
│    70                                                                                            │
│    71                                                                                            │
│    72 def get_dataset(                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/argilla/client/sdk/commons/errors_handler.py:63 in       │
│ handle_response_error                                                                            │
│                                                                                                  │
│   60 │   │   error_type = GenericApiError                                                        │
│   61 │   else:                                                                                   │
│   62 │   │   raise HttpResponseError(response=response)                                          │
│ ❱ 63 │   raise error_type(**error_args)                                                          │
│   64                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationApiError: Argilla server returned an error with http status: 422. Error details: {'code': 
'argilla.api.errors::ValidationError', 'params': {'errors': [{'loc': ['body', 'name'], 'msg': 'string does not 
match regex "^(?!-|_)[a-zA-Z0-9-_ ]+$"', 'type': 'value_error.str.regex', 'ctx': {'pattern': '^(?!-|_)[a-zA-Z0-9-_ 
]+$'}, 'value': {}}]}}

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/

In [7]:
# !pip install 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

model_path = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_path, src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Check if a GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Move the model and tokenizer to the GPU if available
model = model.to(device)

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
def open_translate(texts, trg_lang):
    if isinstance(texts, str):  # If a single text is provided, convert it to a list
        texts = [texts]

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(
        **inputs.to(device), forced_bos_token_id=tokenizer.lang_code_to_id[trg_lang]
    )
    translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

    if len(translations) == 1:
        return translations[0]
    else:
        return translations


example = "We will first start with an example of a translation pipeline with open source models. Even though these models are able to run on CPU it is highly recommended to use a GPU in order to speed up inference."
open_translate(example, "swh_Latn")

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


'Kwanza tutaanza na mfano wa bomba la tafsiri na mifano ya chanzo wazi. Ingawa mifano hii ni uwezo wa kukimbia juu ya CPU ni sana ilipendekeza kutumia GPU ili kuharakisha inference.'

In [22]:
LANGUAGE = 'Swahili'
argilla_ds = rg.FeedbackDataset.from_argilla(f"DIBT Translation for {LANGUAGE}", workspace="admin")
argilla_ds

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/argilla/client/feedback/dataset/local/mixins.py:339 in   │
│ from_argilla                                                                                     │
│                                                                                                  │
│    336 │   │                                                                                     │
│    337 │   │   existing_dataset = get_dataset_by_name_and_workspace(name=name, workspace=worksp  │
│    338 │   │   if existing_dataset is None:                                                      │
│ ❱  339 │   │   │   raise ValueError(                                                             │
│    340 │   │   │   │   f"Could not find a `FeedbackDataset` in Argilla with name='{name}'."      │
│    341 │   │   │   │   if name and not workspace                                                 │
│    342 │   │   │   │   else (                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Could not find a `FeedbackDataset` in Argilla with name='DIBT Translation for Swahili' and 
workspace='admin'.

In [25]:
example = """
The Islamist threat to French schools is taken extremely seriously since the murders of two teachers.
There is indignation in France after the headteacher of a Paris school resigned because of fears for his life.
The head was sent death threats after he insisted that a teenage girl remove her Islamic head-covering inside the school, in accordance with French law.
His name has not been made public. Two people, who had no link with the school, have been detained.
The Islamist threat to French schools is taken extremely seriously since the murders of two teachers.
"""
open_translate(example, "swh_Latn")

'Tisho la Kiislamu kwa shule za Ufaransa limechukuliwa kwa uzito sana tangu mauaji ya walimu wawili. Kuna hasira nchini Ufaransa baada ya mkuu wa shule ya Paris kujiuzulu kwa sababu ya hofu ya maisha yake. Mkuu alipelekwa vitisho vya kifo baada ya yeye kusisitiza kwamba msichana tineja kuondoa kifuniko chake cha kichwa cha Kiislamu ndani ya shule, kulingana na sheria ya Ufaransa. Jina lake halikufunuliwa hadharani. Watu wawili, ambao hawakuwa na uhusiano na shule, wamezuiliwa. Tisho la Kiislamu kwa shule za Ufaransa limechukuliwa kwa uzito sana tangu mauaji ya walimu wawili. '